## Ray Config

In [1]:
import ray
from ray import tune

In [2]:
import time

@ray.remote
def func():
    time.sleep(5)
    return 1

In [3]:
ray.shutdown()

In [4]:
ray.init(num_cpus=4) # one raylet (local machine) with 6 cpus.

2019-08-28 22:05:34,082	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-28_22-05-34_074358_17/logs.
2019-08-28 22:05:34,211	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:53827 to respond...
2019-08-28 22:05:34,342	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:20545 to respond...
2019-08-28 22:05:34,346	INFO services.py:809 -- Starting Redis shard with 0.83 GB max memory.
2019-08-28 22:05:34,396	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-28_22-05-34_074358_17/logs.
2019-08-28 22:05:34,402	INFO services.py:1475 -- Starting the Plasma object store with 1.24 GB memory using /dev/shm.


{'node_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:53827',
 'object_store_address': '/tmp/ray/session_2019-08-28_22-05-34_074358_17/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-28_22-05-34_074358_17/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-28_22-05-34_074358_17'}

## Basic Examples

In [5]:
%%time

# Execute function on Ray and get result.
o_id = func.remote()
ray.get(o_id)

CPU times: user 190 ms, sys: 170 ms, total: 360 ms
Wall time: 5.35 s


1

In [6]:
%%time

# Run 6 instances in parallel, one per core. Serial time = 30 seconds. Parallel ~ 5.
ray.get([func.remote() for _ in range(4)])

CPU times: user 180 ms, sys: 140 ms, total: 320 ms
Wall time: 5.41 s


[1, 1, 1, 1]

## Experiment Runner

In [41]:
class ExampleExperiment(tune.Trainable):
    def _setup(self, config):
        self.config = config

    def _train(self):
        config = self.config
        some_metric = config["alpha"] + config["beta"] + config["list_value"][0]
        result_dict = {"some_metric": some_metric}
        
        if self.iteration == 5:
            result_dict["done"] = tune.result.DONE
            
        return result_dict

In [42]:
analysis = tune.run(
     ExampleExperiment,
     name="example-experiment",
     num_samples=5,
     config={
         "alpha": tune.sample_from(lambda spec: np.random.uniform(1)),
         "beta": tune.sample_from(lambda spec: spec.config.alpha * np.random.normal()),
         "list_value": [
             tune.grid_search([8, 12]),
             tune.grid_search([16, 64]),
         ]
     }
 )

2019-08-28 22:02:44,074	INFO trial_runner.py:176 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 1.3/4.1 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/4 CPUs, 0/0 GPUs
Memory usage on this node: 1.3/4.1 GB
Result logdir: /home/jovyan/ray_results/example-experiment
Number of trials: 20 ({'RUNNING': 1, 'PENDING': 19})
PENDING trials:
 - ExampleExperiment_1_alpha=1.0,beta=2.5322,list_value_0=12,list_value_1=16:	PENDING
 - ExampleExperiment_2_alpha=1.0,beta=-0.251,list_value_0=8,list_value_1=64:	PENDING
 - ExampleExperiment_3_alpha=1.0,beta=-1.9911,list_value_0=12,list_value_1=64:	PENDING
 - ExampleExperiment_4_alpha=1.0,beta=0.016279,list_value_0=8,list_value_1=16:	PENDING
 - ExampleExperiment_5_alpha=1.0,beta=-0.33695,list_value_0=12,list_value_1=16:	PENDING
 - ExampleExperiment_6_alpha=1.0,beta=0.044486,list_value_0=8,list_value_1=64:	PENDING
 - ExampleExperiment_7_alpha=1.0,beta=-0.81658,list_value_0=12,list_value_1=64:	PENDING
 - ExampleExper

Result for ExampleExperiment_7_alpha=1.0,beta=-0.81658,list_value_0=12,list_value_1=64:
  date: 2019-08-28_22-02-47
  done: true
  experiment_id: e69e5e2816fd4363a2661755d9701a1e
  hostname: c3192f238ee8
  iterations_since_restore: 6
  node_ip: 172.17.0.2
  pid: 661
  some_metric: 12.183415997203118
  time_since_restore: 0.00012159347534179688
  time_this_iter_s: 2.2172927856445312e-05
  time_total_s: 0.00012159347534179688
  timestamp: 1567029767
  timesteps_since_restore: 0
  training_iteration: 6
  trial_id: 8feb6828
  
Result for ExampleExperiment_6_alpha=1.0,beta=0.044486,list_value_0=8,list_value_1=64:
  date: 2019-08-28_22-02-47
  done: false
  experiment_id: ace39ab89e8749be8ee2a4f6a58fa3a4
  hostname: c3192f238ee8
  iterations_since_restore: 1
  node_ip: 172.17.0.2
  pid: 658
  some_metric: 9.044485527812212
  time_since_restore: 1.621246337890625e-05
  time_this_iter_s: 1.621246337890625e-05
  time_total_s: 1.621246337890625e-05
  timestamp: 1567029767
  timesteps_since_resto

Result for ExampleExperiment_12_alpha=1.0,beta=0.26713,list_value_0=8,list_value_1=16:
  date: 2019-08-28_22-02-51
  done: false
  experiment_id: 3d5d554ab7d444018bbce857e67f3ac0
  hostname: c3192f238ee8
  iterations_since_restore: 1
  node_ip: 172.17.0.2
  pid: 724
  some_metric: 9.26712926503789
  time_since_restore: 2.288818359375e-05
  time_this_iter_s: 2.288818359375e-05
  time_total_s: 2.288818359375e-05
  timestamp: 1567029771
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8fec99a0
  
Result for ExampleExperiment_12_alpha=1.0,beta=0.26713,list_value_0=8,list_value_1=16:
  date: 2019-08-28_22-02-51
  done: true
  experiment_id: 3d5d554ab7d444018bbce857e67f3ac0
  hostname: c3192f238ee8
  iterations_since_restore: 6
  node_ip: 172.17.0.2
  pid: 724
  some_metric: 9.26712926503789
  time_since_restore: 0.00012493133544921875
  time_this_iter_s: 2.3365020751953125e-05
  time_total_s: 0.00012493133544921875
  timestamp: 1567029771
  timesteps_since_restore: 0
  train

In [43]:
print("Best config is", analysis.get_best_config(metric="some_metric"))

Best config is {'alpha': 1.0, 'beta': 2.532151610979697, 'list_value': [12, 16]}


In [44]:
# Get a dataframe for the last reported results of all of the trials
df = analysis.dataframe(metric="some_metric", mode="max")
df.head()

some_metric   done  timesteps_total  episodes_total  training_iteration  \
0     8.577265  False              NaN             NaN                   1   
1    15.532152  False              NaN             NaN                   1   
2     8.748997  False              NaN             NaN                   1   
3    11.008941  False              NaN             NaN                   1   
4     9.016279  False              NaN             NaN                   1   

                      experiment_id                 date   timestamp  \
0  87be90942c2144bcad63a8622925af74  2019-08-28_22-02-44  1567029764   
1  e6f330d571fa4973934069ed12c168ea  2019-08-28_22-02-44  1567029764   
2  a59f5888fb7b4b5baeeb1fb33c2fce43  2019-08-28_22-02-44  1567029764   
3  8912e95790324c27842cb9163d450a70  2019-08-28_22-02-44  1567029764   
4  48712607b56d4b73af1fcaaa1dfe835e  2019-08-28_22-02-46  1567029766   

   time_this_iter_s  time_total_s  ...      hostname     node_ip  \
0          0.000014      0.000014  ...  c3192f238ee8  172.17.0.2   
1          0.000009      0.000009  ...  c3192f238ee8  172.17.0.2   
2          0.000248      0.000248  ...  c3192f238ee8  172.17.0.2   
3          0.000015      0.000015  ...  c3192f238ee8  172.17.0.2   
4          0.000017      0.000017  ...  c3192f238ee8  172.17.0.2   

  time_since_restore  timesteps_since_restore  iterations_since_restore  \
0           0.000014                        0                         1   
1           0.000009                        0                         1   
2           0.000248                        0                         1   
3           0.000015                        0                         1   
4           0.000017                        0                         1   

   trial_id config:alpha  config:beta  config:list_value  \
0  8fe9a6e6          1.0    -0.422735            [8, 16]   
1  8fe9d6a2          1.0     2.532152           [12, 16]   
2  8fea0a6e          1.0    -0.251003            [8, 64]   
3  8fea35de          1.0    -1.991059           [12, 64]   
4  8fea72b0          1.0     0.016279            [8, 16]   

                                              logdir  
0  /home/jovyan/ray_results/example-experiment/Ex...  
1  /home/jovyan/ray_results/example-experiment/Ex...  
2  /home/jovyan/ray_results/example-experiment/Ex...  
3  /home/jovyan/ray_results/example-experiment/Ex...  
4  /home/jovyan/ray_results/example-experiment/Ex...  

[5 rows x 21 columns]

In [45]:
trials = analysis.trials # list of all trials
all_dataframes = analysis.trial_dataframes # df of iterations for each trial